In [1]:
%autoreload 2

In [2]:
from collections import defaultdict
from argparse import Namespace
import copy
import gzip
import itertools
import os
import pickle
import sys
import typing

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tatsu
import tatsu.ast
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import tqdm
import sklearn
from sklearn.model_selection import GridSearchCV, train_test_split, KFold
from sklearn.pipeline import Pipeline

sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../src'))
from src import fitness_energy_utils as utils
from src.fitness_energy_utils import NON_FEATURE_COLUMNS
from src.fitness_features import build_fitness_featurizer, ASTFitnessFeaturizer
from src.ast_counter_sampler import *
from src.ast_mcmc_regrowth import *

# Plan of attack
* Use the code in `ast_counter_sampler.py` to generate a sample from the MLE
* Score it with a fitness function adapted to working with a single example at a time
* At each subsequent iteration, regrow the game from a random node, and score the regrowth
* Since we have energy scores before and after, we can either accept greedily if $E_{new} < E_{old}$, or accept with probability $\exp(\beta (E_{old} - E_{new} ))$

In [3]:
DEFAULT_ARGS = argparse.Namespace(
    grammar_file=os.path.join('..', DEFAULT_GRAMMAR_FILE),
    parse_counter=False,
    counter_output_path=os.path.join('..', DEFAULT_COUNTER_OUTPUT_PATH),
    random_seed=DEFUALT_RANDOM_SEED,
)

In [10]:
mcmc = MCMCRegrowthSampler(DEFAULT_ARGS, '../models/cv_fitness_model_2022_12_26.pkl.gz', greedy_acceptance=True, 
    plateau_patience_steps=20, acceptance_temperature=10.0, max_steps=1000)

In [11]:
# %prun -r 
mcmc.multiple_samples(10, verbose=0, should_tqdm=True)
print([x[2] for x in mcmc.samples])

  0%|          | 0/10 [00:00<?, ?it/s]

[0.1560910940170288, 0.2381468266248703, 1.3464714288711548, 1.019278645515442, 2.6622514724731445, 0.3673384487628937, 0.6889153718948364, 0.871972918510437, 2.2208845615386963, 4.490621089935303]


In [ ]:
print(ast_printer.ast_to_string(mcmc.samples[1][0], '\n'))

In [ ]:
fitness_featurizer = build_fitness_featurizer(DEFAULT_ARGS)
features = fitness_featurizer.parse(mcmc.samples[1][0], 'mcmc', True)
{k: v for k, v in features.items() if 'variables' in k}
# features

In [ ]:
with open('/Users/guydavidson/Downloads/samples.pkl', 'rb') as f:
    samples = pickle.load(f)

print([x[2] for x in samples])

In [ ]:
print(ast_printer.ast_to_string(samples[8][0], '\n'))

In [ ]:
fitness_featurizer = build_fitness_featurizer(DEFAULT_ARGS)
features = fitness_featurizer.parse(samples[7][0], 'mcmc', True)
{k: v for k, v in features.items() if 'correctly' in k or 'forall' in k or 'no' in k}

In [ ]:
with gzip.open(DEFAULT_FITNESS_FUNCTION_PATH, 'rb') as f:
    cv_fitness_model = pickle.load(f)

In [ ]:
weights = cv_fitness_model.named_steps['fitness'].model.fc1.weight.data.detach().numpy().squeeze()
bias = cv_fitness_model.named_steps['fitness'].model.fc1.bias.data.detach().numpy().squeeze()
print(weights.mean(), bias)

In [ ]:
plt.hist(weights, bins=100)
plt.title('Fitness Model Weights')
plt.xlabel('Weight magnitude')
plt.ylabel('Count')

In [12]:
import ast_crossover_sampler
asts = [ast for ast in cached_load_and_parse_games_from_file('../dsl/interactive-beta.pddl', 
    mcmc.grammar_parser, False, relative_path='..')]

mcmc_with_crossover = MCMCRegrowthCrossoverSampler(DEFAULT_ARGS, 
    ast_crossover_sampler.CrossoverType.SAME_RULE, asts, 0.25,
    '../models/cv_fitness_model_2022_12_26.pkl.gz', 
    greedy_acceptance=True, plateau_patience_steps=20, acceptance_temperature=10.0, max_steps=1000)

In [13]:
mcmc_with_crossover.multiple_samples(10, verbose=2, should_tqdm=True)
print([x[2] for x in mcmc_with_crossover.samples])

  0%|          | 0/10 [00:00<?, ?it/s]

Sampling exception, skipping sample
Regrowth generated identical games, repeating
Accepted step 1 with fitness 7.857532501220703
Accepted step 2 with fitness 7.784302711486816
Accepted step 3 with fitness 7.775148391723633
Accepted step 4 with fitness 7.692765235900879
Accepted step 6 with fitness 7.536055564880371
Regrowth generated identical games, repeating
Accepted step 8 with fitness 7.5235395431518555
Accepted step 10 with fitness 7.4317522048950195
Accepted step 11 with fitness 6.490839004516602
Accepted step 17 with fitness 5.810407638549805
Accepted step 19 with fitness 5.400604248046875
Accepted step 28 with fitness 5.354710578918457
Accepted step 44 with fitness 4.757689476013184
Accepted step 49 with fitness 4.688848495483398
Accepted step 68 with fitness 4.574114799499512
Accepted step 75 with fitness 4.34464693069458
Regrowth generated identical games, repeating
Accepted step 93 with fitness 3.9308552742004395
Regrowth generated identical games, repeating
Accepted step 10

In [14]:
print([x[2] for x in mcmc_with_crossover.samples])

[-0.5954062938690186, -0.4696420133113861, 1.7651313543319702, 5.2624006271362305, 0.8746525049209595, 1.2288413047790527, 0.9395236968994141, 1.8448952436447144, 0.8919482231140137, 1.2082358598709106]
